In [1]:
#import cell
import QuantLib as ql
import numpy
from collateralAgreement.collateralAgreement import CollateralAgreement, Margining, Clearing, Tradecount, Dispute, \
    InitialMargining
from instruments.interestRateInstrument.irs import IRS
from jupyterUtils import export
from marketdata import init_marketdata
from marketdata.interestRateIndices import InterestRateIndex
from sa_ccr.sa_ccr import SA_CCR
from utilities.Enums import SwapDirection
asdf =1

### Risk Horizon

For unmargined transaction the margining factor is


$$MF^{\text{unmargined}}_i = \sqrt{\frac{\min\left(M_i;1\text{ year}\right)}{1\text{ year}}}$$

This factor can be used to scale down a risk weight calibrated for a 1 year horizon to a shorter period.

With margining the margin period of risk (MPOR) is:

* 10 business days for small, uncleared OTC portfolios
* 5 business days for cleared derivatives
* 20 business days for netting sets with more than 5000 transactions that are not with a central counterparty
* and doubling this period for portfolios with outstanding disputes

The margining factor is then

$$ MF^{\text{margined}}_i = \frac{3}{2}\sqrt{\frac{MPOR_i}{1\text{ year}}} $$

At this point we need to introduce a collateral agreement object. For simplicities sake we will not differentiate between collateral and netting sets in this thesis. All trades that are covered by the same collateral agreement are also admissible for netting with each other. (Also refer to the introduction of close out netting above). To take into account the different parameters determining the risk horizon a couple of parameters are required to create a collateral agreement. As an example, below we are setting up a collateral agreement for uncleared derivatives without exchange of variation margin or initial margin.


In [2]:
ca = CollateralAgreement(
        margining=Margining.UNMARGINED,
        clearing=Clearing.UNCLEARED,
        initialMargining=InitialMargining.NO_IM,
        tradecount=Tradecount.UNDER_FIVE_THOUSAND,
        dispute=Dispute.NO_OUTSTANDING_DISPUTES,
        threshold=0.0,      #Threshold to trigger a margin call
        mta=0.0,            #received initial margin
        )

With this collateral set object we can define a function for calculation the margining factor:

For trades of differing maturity let's compare the margining factor for the three most common scenarios:

1. No margining
2. Bilateral margining
3. Centrally cleared

In [3]:

three_day = IRS(notional = 1000000, timeToSwapStart=ql.Period(2, ql.Days), timeToSwapEnd=ql.Period(3, ql.Days), swapDirection=SwapDirection.PAYER, index=InterestRateIndex.EURIBOR6M)
two_weeks = IRS(notional = 1000000, timeToSwapStart=ql.Period(2, ql.Days), timeToSwapEnd=ql.Period(2, ql.Weeks), swapDirection=SwapDirection.PAYER, index=InterestRateIndex.EURIBOR6M)
six_months = IRS(notional = 1000000, timeToSwapStart=ql.Period(2, ql.Days), timeToSwapEnd=ql.Period(6, ql.Months), swapDirection=SwapDirection.PAYER, index=InterestRateIndex.EURIBOR6M)
one_year = IRS(notional = 1000000, timeToSwapStart=ql.Period(2, ql.Days), timeToSwapEnd=ql.Period(1, ql.Years), swapDirection=SwapDirection.PAYER, index=InterestRateIndex.EURIBOR6M)
ten_years = IRS(notional = 1000000, timeToSwapStart=ql.Period(2, ql.Days), timeToSwapEnd=ql.Period(10, ql.Years), swapDirection=SwapDirection.PAYER, index=InterestRateIndex.EURIBOR6M)

no_margining = CollateralAgreement(margining=Margining.UNMARGINED, initialMargining=InitialMargining.NO_IM)
bilateral_margining = CollateralAgreement(margining = Margining.MARGINED, initialMargining=InitialMargining.SIMM)
central_clearing = CollateralAgreement(margining = Margining.MARGINED, clearing = Clearing.CLEARED, initialMargining=InitialMargining.CCPIMM)

trades = [three_day, two_weeks, six_months, one_year, ten_years]
cas = [no_margining, bilateral_margining, central_clearing]
ar = numpy.empty([3,5])
i = 0
for t in trades:
    j = 0
    for ca in cas:
        ar[j,i] = '%.4f' % SA_CCR.margining_factor(t,ca)
        j += 1
    i += 1

In [4]:
# making a nice looking latex pandas dataframe out of it

import pandas as pd

pd.set_option('display.notebook_repr_html', True)

def _repr_latex_(self):
    return "\centering{%s}" % self.to_latex()

pd.DataFrame._repr_latex_ = _repr_latex_  # monkey patch pandas DataFrame

pd.DataFrame(index=['No margining', 'Bilateral margining', 'Centrally cleared'],
          columns=['Three days', 'Two weeks', 'Six months', 'One year', 'Ten years'],
          data = ar)

,Three days,Two weeks,Six months,One year,Ten years
No margining,0.2000,0.2000,0.7071,1.0000,1.0000
Bilateral margining,0.3000,0.3000,0.3000,0.3000,0.3000
Centrally cleared,0.2121,0.2121,0.2121,0.2121,0.2121


In [5]:
export("building_SA_CCR_risk_horizon.ipynb")
